In [132]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit

In [133]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t, anisotropic = []):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix

        for i in range(0, len(self.sig_scattering_matrix)):
            self.sig_s.append(sum(self.sig_scattering_matrix[i]))
            
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        self.anisotropic = anisotropic
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_anisotropic(self, energy_group_idx, energy_sub_group):
        return self.anisotropic[energy_group_idx][energy_sub_group]
        
        

In [134]:
energy_groups_2 = [0, 6.25, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [135]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self): 
        
        return self.energy_group
     #   res = next(x for x, val in enumerate(self.energy_groups)
      #                            if val > self.energy)        
       # return res - 1
    
    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [136]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [137]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [138]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [139]:
class Flux_estimator:
    def __init__(self, universe, group):
        self.boundaries = universe
        self.collision_sum = [0]
        self.group = group
        
    def add_collision(self, particle):
        if (self.boundaries.is_inside(particle) and particle.get_energy_group() == self.group):
            self.collision_sum[-1]  += particle.get_weight()
            

In [140]:
import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [141]:
def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [142]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [143]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [144]:
def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False

In [145]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [146]:
def calculate_mu(anisotropic_coeffs):
    
    
    b = 3. * anisotropic_coeffs[1] / anisotropic_coeffs[0]
    gamma =  random.uniform(0, 1)
    sqr = (1/2 - b)**2 + 2 * b * gamma
    

    result_mu1 = (-1/2 + math.sqrt(sqr))/b
    result_mu2 = (-1/2 - math.sqrt(sqr))/b
  #  if abs(result_mu1) < 1:
   #     return result_mu1
    
   # if abs(result_mu2) < 1:
    #    return result_mu2
    
    
    return anisotropic_coeffs[1]/ anisotropic_coeffs[0]

In [176]:
def calculate_mu_qvadric(anisotropic_coeffs):
    
    b = 3. * anisotropic_coeffs[1] / anisotropic_coeffs[0]
    gamma =  random.uniform(0, 1)
    sqr = (1/2 - b)**2 + 2 * b * gamma
    

    result_mu1 = (-1/2 + math.sqrt(sqr))/b
    result_mu2 = (-1/2 - math.sqrt(sqr))/b
    if abs(result_mu1) < 1:
        return result_mu1
    
    if abs(result_mu2) < 1:
        return result_mu2

In [177]:
def get_anisotropic_direction_2(particle, mu):
    
    
    dir_x = particle.direction.tetta_x
    dir_y = particle.direction.tetta_y
    dir_z = particle.direction.tetta_z
    
    b = math.sqrt(1 - mu * mu)
    ro = math.sqrt(1 - dir_z * dir_z)
    phi = random.uniform(-math.pi, math.pi)

    c = math.cos(phi)
    d = math.sin(phi)
    a = mu
    
    dir_x_tmp = (b * c * dir_x * dir_z)/ro  - (b * d * dir_y)/ro + a * dir_x    
    dir_y_tmp = (b * c * dir_y * dir_z)/ro  + (b * d * dir_x)/ro + a * dir_y  
    dir_z_tmp = -b * c * ro  + a * dir_z
    
    particle.direction.tetta_x = dir_x_tmp
    particle.direction.tetta_y = dir_y_tmp
    particle.direction.tetta_z = dir_z_tmp

In [178]:
def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx_new = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx_new)
    anisotropic_coeffs = c_s.get_anisotropic(energy_group_idx, energy_group_idx_new)
    mu = calculate_mu(anisotropic_coeffs)
  #  set_random_direction(particle)
    get_anisotropic_direction_2(particle, mu)


In [179]:
def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    set_random_direction(particle)
    

In [180]:
def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        process_scattering(particle, c_s)
        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        

In [181]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [182]:
def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    
    while not particle.is_terminated():
     #   set_random_direction(particle)
        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions

In [183]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [184]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
     #   set_random_direction(particles[i])       

In [185]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [186]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [187]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [188]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [189]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [190]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   

In [191]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [192]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [193]:
cs_fission_rr = [0.050632, 0.0010484]

cs_capture_rr = [0.025788, 0.0010046]

cs_scattering_rr = [[2.44383, 0.0], [0.029227, 0.62568]]

cs_total_rr = [2.52025, 0.65696]

cs_production_neutrons_rr = [2.50, 2.50]

cs_xi_table_rr = [ 0., 1.]
cs_anisotropic =[[[2.44383, 0.83318], [0., 0,]], [[0.029227, 0.0075737], [0.62568, 0.27459]]]


research_reactor_b = Sig(2, cs_fission_rr, cs_capture_rr, cs_scattering_rr, 
                       cs_production_neutrons_rr,  cs_xi_table_rr, cs_total_rr, cs_anisotropic)


energy_groups_2 = [0, 2 * 1E10]

In [194]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 9.4959)

y_m5 = Plane(0, 1, 0, -9.4959)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective", "black", "black", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -9.4959, -1000, 1000, 9.4959, 1000 )
slab.set_zero_point(0., 0., 0.)

In [195]:
estimators = []

In [196]:
cells = [slab]

In [197]:
materials = [research_reactor_b]

In [198]:
U235_Reactor = Universe(cells, materials)

In [202]:
test_number_of_particles = 10
test_number_interations = 1000
test_number_inactive = 10

In [ ]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, research_reactor_b, estimators)

 num1  9
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
 keff_cycle , k_effective_exp, std_k_effective 0.975436232517671   1.0081202577904773  0.03268402527280628
i == 13
 keff_cycle , k_effective_exp, std_k_effective 1.0116185166758414   1.0092863440855986  0.018906125813048437
i == 14
 keff_cycle , k_effective_exp, std_k_effective 1.0172280588682352   1.0112717727812577  0.01351527741982957
i == 15
 keff_cycle , k_effective_exp, std_k_effective 1.0080980815863563   1.0106370345422775  0.010488113599356025
i == 16
 keff_cycle , k_effective_exp, std_k_effective 0.9527971387654748   1.0009970519128104  0.012894299115439566
i == 17
 keff_cycle , k_effective_exp, std_k_effective 1.0194361556782414   1.0036312095935862  0.011211513566753354
i == 18
 keff_cycle , k_effective_exp, std_k_effective 1.0597807941804278   1.0106499076669415  0.011980636460056737
i == 19
 keff_cycle , k_effective_exp, std_k_effective 1.0081906237448657   1.01037665389

 keff_cycle , k_effective_exp, std_k_effective 1.00086345583458   0.996158225541412  0.00482401509121997
i == 83
 keff_cycle , k_effective_exp, std_k_effective 1.0097340554216006   0.9963441958137433  0.004761107178709364
i == 84
 keff_cycle , k_effective_exp, std_k_effective 1.0114013598539027   0.9965476710034753  0.004700733054025321
i == 85
 keff_cycle , k_effective_exp, std_k_effective 1.0185901264344313   0.9968415704092214  0.004646936374935492
i == 86
 keff_cycle , k_effective_exp, std_k_effective 1.0478175703434378   0.997512307250461  0.004634181908377039
i == 87
 keff_cycle , k_effective_exp, std_k_effective 0.9386138335694562   0.9967473920078507  0.004637125015160879
i == 88
 keff_cycle , k_effective_exp, std_k_effective 1.0305034521309715   0.9971801620094292  0.0045977017238915925
i == 89
 keff_cycle , k_effective_exp, std_k_effective 0.9548050279997635   0.9966437679080411  0.0045707131535804415
i == 90
 keff_cycle , k_effective_exp, std_k_effective 0.9847686282175553  

 keff_cycle , k_effective_exp, std_k_effective 1.0170141630156373   0.9981036511339496  0.003435274260403847
i == 153
 keff_cycle , k_effective_exp, std_k_effective 0.9394023908708597   0.9976931528104316  0.0034357775886783787
i == 154
 keff_cycle , k_effective_exp, std_k_effective 1.0252600590412475   0.9978845896592566  0.003417201102896219
i == 155
 keff_cycle , k_effective_exp, std_k_effective 0.9709691585883173   0.9976989659966984  0.003398625283842891
i == 156
 keff_cycle , k_effective_exp, std_k_effective 0.9112239574283527   0.997106671417463  0.003426840900019639
i == 157
 keff_cycle , k_effective_exp, std_k_effective 1.0009877846167865   0.9971330735480708  0.0034035516263796767
i == 158
 keff_cycle , k_effective_exp, std_k_effective 1.0111173606607455   0.9972275619745077  0.0033817967094162548
i == 159
 keff_cycle , k_effective_exp, std_k_effective 1.0128151717393916   0.9973321768051445  0.003360652097732338
i == 160
 keff_cycle , k_effective_exp, std_k_effective 1.01278

 keff_cycle , k_effective_exp, std_k_effective 1.0067554514204937   0.9983105564375133  0.002796672242350822
i == 223
 keff_cycle , k_effective_exp, std_k_effective 0.9810273747322233   0.9982294147393664  0.002784693782005934
i == 224
 keff_cycle , k_effective_exp, std_k_effective 0.9745681551373214   0.998118848105712  0.002773855132399397
i == 225
 keff_cycle , k_effective_exp, std_k_effective 1.0146504742858624   0.998195739390271  0.0027619938340218246
i == 226
 keff_cycle , k_effective_exp, std_k_effective 1.069112386242875   0.9985240571997737  0.0027687121999219622
i == 227
 keff_cycle , k_effective_exp, std_k_effective 1.0781151474607213   0.998890836417566  0.0027802233737451937
i == 228
 keff_cycle , k_effective_exp, std_k_effective 0.9989309670289687   0.9988910205029394  0.002767440675957642
i == 229
 keff_cycle , k_effective_exp, std_k_effective 1.0275394693049127   0.9990218353376517  0.0027578792024713287
i == 230
 keff_cycle , k_effective_exp, std_k_effective 1.0530952

 keff_cycle , k_effective_exp, std_k_effective 1.0401439981970124   0.9991268637058588  0.002463923006499888
i == 293
 keff_cycle , k_effective_exp, std_k_effective 1.0184288540888   0.9991950686188729  0.002456148303341209
i == 294
 keff_cycle , k_effective_exp, std_k_effective 0.9751454038798796   0.9991103867007778  0.0024489491547448916
i == 295
 keff_cycle , k_effective_exp, std_k_effective 1.0810104936700384   0.9993977554971611  0.002457202903144502
i == 296
 keff_cycle , k_effective_exp, std_k_effective 0.9750899598725231   0.9993127632047674  0.0024500708350857175
i == 297
 keff_cycle , k_effective_exp, std_k_effective 0.9828878309958928   0.9992555334758165  0.0024421897247606976
i == 298
 keff_cycle , k_effective_exp, std_k_effective 1.037210671375481   0.9993873221490792  0.002437260802836998
i == 299
 keff_cycle , k_effective_exp, std_k_effective 0.9998618529683307   0.9993889641242323  0.0024288132880612052
i == 300
 keff_cycle , k_effective_exp, std_k_effective 1.0147933

...

 keff_cycle , k_effective_exp, std_k_effective 0.9335988854737863   0.9997625753396852  0.001933725654649435
i == 433
 keff_cycle , k_effective_exp, std_k_effective 0.9662319005729343   0.999683306605012  0.0019307766723230767
i == 434
 keff_cycle , k_effective_exp, std_k_effective 0.9897055000855685   0.9996597740424661  0.0019263613141684366
i == 435
 keff_cycle , k_effective_exp, std_k_effective 0.9767175561976797   0.9996057923534196  0.0019225813448726888
i == 436
 keff_cycle , k_effective_exp, std_k_effective 0.9586568950221088   0.9995096681812803  0.0019204700658077092
i == 437
 keff_cycle , k_effective_exp, std_k_effective 1.0197813084350305   0.9995571427486193  0.0019165552801979132
i == 438
 keff_cycle , k_effective_exp, std_k_effective 0.9816503486800326   0.9995153044447208  0.001912529783780857
i == 439
 keff_cycle , k_effective_exp, std_k_effective 0.9971562794452997   0.9995098055519481  0.0019080743879806021
i == 440
 keff_cycle , k_effective_exp, std_k_effective 0.94

 keff_cycle , k_effective_exp, std_k_effective 1.014707378334224   0.9989668872022909  0.001777071233129055
i == 503
 keff_cycle , k_effective_exp, std_k_effective 0.9488613080881793   0.9988652531675766  0.0017763728093908315
i == 504
 keff_cycle , k_effective_exp, std_k_effective 0.9966006518262498   0.9988606689543351  0.0017727791932051868
i == 505
 keff_cycle , k_effective_exp, std_k_effective 0.9669505523148646   0.9987962040722351  0.0017703682738274662
i == 506
 keff_cycle , k_effective_exp, std_k_effective 1.0441799185308116   0.9988877034965468  0.001769163093379215
i == 507
 keff_cycle , k_effective_exp, std_k_effective 0.9571680325355956   0.9988037604966253  0.0017675941712005413
i == 508
 keff_cycle , k_effective_exp, std_k_effective 0.9978042343976099   0.9988017534161053  0.001764042356302044
i == 509
 keff_cycle , k_effective_exp, std_k_effective 0.956962685440854   0.9987179075884995  0.0017624991436146742
i == 510
 keff_cycle , k_effective_exp, std_k_effective 0.9792

 keff_cycle , k_effective_exp, std_k_effective 1.019410582063262   0.9984987800867832  0.001657690533373913
i == 573
 keff_cycle , k_effective_exp, std_k_effective 0.9579913705531689   0.998426830869139  0.0016563069841072401
i == 574
 keff_cycle , k_effective_exp, std_k_effective 0.9769038871211655   0.9983886696213589  0.001653808001227871
i == 575
 keff_cycle , k_effective_exp, std_k_effective 1.0108065749525876   0.9984106482148654  0.001651024609501486
i == 576
 keff_cycle , k_effective_exp, std_k_effective 0.989340744619311   0.9983946236502089  0.0016481829252410897
i == 577
 keff_cycle , k_effective_exp, std_k_effective 1.0448860490247471   0.9984766191094232  0.0016473154449810797
i == 578
 keff_cycle , k_effective_exp, std_k_effective 0.9811848092831311   0.9984461757822644  0.001644694461665452
i == 579
 keff_cycle , k_effective_exp, std_k_effective 1.0367026281933815   0.9985134103207723  0.001643177524475105
i == 580
 keff_cycle , k_effective_exp, std_k_effective 1.0413901

 keff_cycle , k_effective_exp, std_k_effective 1.0378524083615728   0.9989444434560354  0.0015431003888795743
i == 643
 keff_cycle , k_effective_exp, std_k_effective 0.9991414796643896   0.9989447547296664  0.0015406607343891967
i == 644
 keff_cycle , k_effective_exp, std_k_effective 1.017278129225712   0.9989736717241396  0.0015385005303588716
i == 645
 keff_cycle , k_effective_exp, std_k_effective 1.042393100235272   0.999042048776913  0.0015375969029351928
i == 646
 keff_cycle , k_effective_exp, std_k_effective 1.0020144729903167   0.9990467223998901  0.0015351845081787955
i == 647
 keff_cycle , k_effective_exp, std_k_effective 0.9924451415379988   0.9990363588506564  0.001532807625909196
i == 648
 keff_cycle , k_effective_exp, std_k_effective 0.9568230493716281   0.9989701937887772  0.0015318328342702402
i == 649
 keff_cycle , k_effective_exp, std_k_effective 0.9420292380014172   0.9988810843118017  0.0015320274168734702
i == 650
 keff_cycle , k_effective_exp, std_k_effective 1.056

 keff_cycle , k_effective_exp, std_k_effective 1.0123765578891548   0.9990255007469301  0.0014708606975319023
i == 713
 keff_cycle , k_effective_exp, std_k_effective 1.0161231082345272   0.9990498216679651  0.001468968292616974
i == 714
 keff_cycle , k_effective_exp, std_k_effective 1.0530924501919323   0.9991265867653002  0.001468887476461652
i == 715
 keff_cycle , k_effective_exp, std_k_effective 1.0106849238593303   0.999142981569689  0.0014668940898235833
i == 716
 keff_cycle , k_effective_exp, std_k_effective 0.9753807335924515   0.9991093239946504  0.001465201492616133
i == 717
 keff_cycle , k_effective_exp, std_k_effective 1.0385993834595122   0.9991651798071891  0.0014641933812559532
i == 718
 keff_cycle , k_effective_exp, std_k_effective 0.9895254295958732   0.999151564340789  0.001462187241965405
i == 719
 keff_cycle , k_effective_exp, std_k_effective 0.9858685319186845   0.9991328294572599  0.0014602436511415731
i == 720
 keff_cycle , k_effective_exp, std_k_effective 1.01958

In [201]:
difference = (1. - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -107.51818544301184  with standart deviation [pcm]+- 391.4876831350764


In [39]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [40]:
file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)